In [6]:
library(ggplot2)
library(dplyr)
library(geometry)  # For convex hull
library(Seurat)
library(sp)        # For point-in-polygon test

# ----------------------
# 1) Process an ROI CSV file that may contain multiple ROI regions
# ----------------------
process_roi_file <- function(roi_file) {
  # Read CSV (expects columns: x, y, roi_name, slide_id)
  coords <- read.csv(roi_file)
  coords$x <- as.numeric(coords$x)
  coords$y <- as.numeric(coords$y)
  
  # Group by unique combination of roi_name and slide_id
  roi_groups <- coords %>%
    group_by(roi_name, slide_id) %>%
    group_split()
  
  roi_info_list <- list()
  
  for (group in roi_groups) {
    # Get the unique roi_name and slide_id for this group
    current_roi <- unique(group$roi_name)
    current_slide <- unique(group$slide_id)
    
    if (length(current_roi) != 1 || length(current_slide) != 1) {
      stop("Grouping error: expected one unique 'roi_name' and one unique 'slide_id' in each group.")
    }
    
    # Parse the ROI label from the roi_name (expected pattern: e.g., "A2_C1")
    parts <- strsplit(current_roi, "_")[[1]]
    if (length(parts) != 2) {
      stop("roi_name does not match the expected pattern 'SLIDE_ROISECTION': ", current_roi)
    }
    
    # parts[2] should contain the ROI letter and the section number (e.g., "C1")
    roi_section_part <- parts[2]
    roi_val <- sub("\\d+$", "", roi_section_part)    # Remove trailing digits, yields "C"
    section_val <- sub("\\D+", "", roi_section_part)   # Remove non-digits, yields "1"
    
    # Compute the convex hull for this group
    hull_indices <- chull(group$x, group$y)
    hull_coords <- group[hull_indices, ]
    # Close the polygon by appending the first coordinate to the end
    hull_coords <- rbind(hull_coords, hull_coords[1, ])
    
    roi_info_list[[length(roi_info_list) + 1]] <- list(
      hull_coords = hull_coords,
      ROI = roi_val,
      Section = section_val,
      Slide = current_slide
    )
  }
  
  return(roi_info_list)
}

# ----------------------
# 2) Assign ROI information to the Seurat object's metadata
# ----------------------
assign_roi_to_seurat <- function(seurat_object, roi_info) {
  # Extract ROI info: hull coordinates, ROI, Section, Slide
  hull_coords <- roi_info$hull_coords
  roi_val     <- roi_info$ROI
  section_val <- roi_info$Section
  slide_val   <- roi_info$Slide
  
  # Get the Seurat metadata
  metadata <- seurat_object@meta.data
  
  # Ensure that the metadata has cell coordinates columns (x_slide_mm, y_slide_mm)
  if (!("x_slide_mm" %in% colnames(metadata) && "y_slide_mm" %in% colnames(metadata))) {
    stop("Metadata does not contain 'x_slide_mm' and 'y_slide_mm' columns.")
  }
  
  # Create new columns if they do not already exist
  if (!"ROI" %in% colnames(metadata)) {
    metadata$ROI <- NA
  }
  if (!"Section" %in% colnames(metadata)) {
    metadata$Section <- NA
  }
  if (!"Slide" %in% colnames(metadata)) {
    metadata$Slide <- NA
  }
  
  # Perform a point-in-polygon test for each cell using sp::point.in.polygon
  in_roi <- sp::point.in.polygon(
    point.x = metadata$x_slide_mm,
    point.y = metadata$y_slide_mm,
    pol.x   = hull_coords$x,
    pol.y   = hull_coords$y
  )
  
  # Update the metadata: for cells inside the ROI (in_roi == 1), assign values
  metadata$ROI[in_roi == 1]     <- roi_val
  metadata$Section[in_roi == 1] <- section_val
  metadata$Slide[in_roi == 1]   <- slide_val
  
  # Write the updated metadata back to the Seurat object
  seurat_object@meta.data <- metadata
  return(seurat_object)
}

# ----------------------
# 3) Main function: Process ROI CSV file for a given genotype and update the Seurat object
# ----------------------
process_all_rois <- function(genotype) {
  # Construct the file path for the ROI CSV (e.g., "A2_rois.csv" or "B1_rois.csv")
  roi_dir  <- "/Users/katherineridley/Projects/CosMx/APP/region_rois/"
  roi_file <- file.path(roi_dir, paste0(genotype, "_rois.csv"))
  
  if (!file.exists(roi_file)) {
    stop("ROI file not found: ", roi_file)
  }
  
  # Load the corresponding Seurat object
  seurat_file <- paste0("/Users/katherineridley/Projects/CosMx/APP/seuratObject_", genotype, ".RDS")
  if (!file.exists(seurat_file)) {
    stop("Seurat file not found: ", seurat_file)
  }
  seurat_object <- readRDS(seurat_file)
  
  cat("Processing ROI file:", roi_file, "\n")
  
  # Process the ROI file into a list of ROI regions
  roi_info_list <- process_roi_file(roi_file)
  
  # Loop through each ROI region and update the Seurat metadata
  for (roi_info in roi_info_list) {
    seurat_object <- assign_roi_to_seurat(seurat_object, roi_info)
  }
  
  # Save the updated Seurat object
  output_seurat_file <- paste0("/Users/katherineridley/Projects/CosMx/APP/seuratObject_with_ROI_", genotype, ".RDS")
  saveRDS(seurat_object, output_seurat_file)
  cat("Updated Seurat object saved for genotype:", genotype, "\n")
  
  return(seurat_object)
}

# ----------------------
# 4) (Optional) Function to filter/subset Seurat object by region (e.g., cortex or hippocampus)
# ----------------------
save_seurat_by_region <- function(seurat_object, genotype, region_pattern, region_name) {
  if (!"ROI" %in% colnames(seurat_object@meta.data)) {
    stop("ROI column not found in the metadata.")
  }
  
  # Filter cells that match the specified pattern (e.g., "C" or "H")
  region_cells <- seurat_object@meta.data %>%
    filter(grepl(region_pattern, .data$ROI)) %>%
    rownames()
  
  # Ensure that the cells are present in the Seurat object's cell names
  region_cells <- intersect(region_cells, colnames(seurat_object))
  
  # Subset the Seurat object using the intersection of cell names
  seurat_region <- subset(seurat_object, cells = region_cells)
  
  output_file <- paste0("/Users/katherineridley/Projects/CosMx/APP/seuratObject_",
                        genotype, "_", region_name, ".RDS")
  saveRDS(seurat_region, output_file)
  cat("Saved", region_name, "Seurat object for genotype:", genotype, "\n")
  
  return(seurat_region)
}

# ----------------------
# 5) Run processing for genotypes B1 and A2
# ----------------------
# This will look for:
#   /Users/katherineridley/Projects/CosMx/APP/region_rois/B1_rois.csv
#   /Users/katherineridley/Projects/CosMx/APP/region_rois/A2_rois.csv
# and the corresponding Seurat files:
#   /Users/katherineridley/Projects/CosMx/APP/seuratObject_B1.RDS
#   /Users/katherineridley/Projects/CosMx/APP/seuratObject_A2.RDS

seurat_B1 <- process_all_rois("B1")
seurat_A2 <- process_all_rois("A2")

# Optionally, subset the Seurat object based on region patterns (e.g., "C" for cortex, "H" for hippocampus)
save_seurat_by_region(seurat_B1, "B1", "C", "Cortex")
save_seurat_by_region(seurat_B1, "B1", "H", "Hippocampus")

save_seurat_by_region(seurat_A2, "A2", "C", "Cortex")
save_seurat_by_region(seurat_A2, "A2", "H", "Hippocampus")


Processing ROI file: /Users/katherineridley/Projects/CosMx/APP/region_rois//B1_rois.csv 
Updated Seurat object saved for genotype: B1 
Processing ROI file: /Users/katherineridley/Projects/CosMx/APP/region_rois//A2_rois.csv 
Updated Seurat object saved for genotype: A2 


ERROR: Error in ..subscript.2ary(x, l[[1L]], l[[2L]], drop = drop[1L]): invalid class "dgCMatrix" object: length of Dimnames[[1]] (134767) is not equal to Dim[1] (313521)


In [5]:
head(seurat_B1@meta.data)


,RNA_nbclust_9e6a7947.d228.4e0b.9639.c721fd9e6b0a_1_clusters,RNA_nbclust_9e6a7947.d228.4e0b.9639.c721fd9e6b0a_1_posterior_probability,fov,Area,AspectRatio,x_FOV_px,y_FOV_px,Width,Height,Mean.Histone,...,RNA_spatialclust_5321a228.edc1.478e.be8b.8f3a09b0c12b_1_neighbours_g,RNA_spatialclust_5321a228.edc1.478e.be8b.8f3a09b0c12b_1_neighbours_h,RNA_spatialclust_5321a228.edc1.478e.be8b.8f3a09b0c12b_1_neighbours_i,RNA_spatialclust_5321a228.edc1.478e.be8b.8f3a09b0c12b_1_neighbours_j,RNA_spatialclust_5321a228.edc1.478e.be8b.8f3a09b0c12b_1_neighbours_k,RNA_spatialclust_5321a228.edc1.478e.be8b.8f3a09b0c12b_1_neighbours_l,spatialclust_5321a228.edc1.478e.be8b.8f3a09b0c12b_1_assignments,ROI,Section,Slide
,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
c_2_100_108,f,1.0000000,100,5365,0.82,3278,457,80,98,368,...,0,0,1,0,0,9,niche5,C,4,B1
c_2_100_109,f,1.0000000,100,12541,0.79,3837,510,146,185,17,...,1,0,1,0,0,5,niche3,C,4,B1
c_2_100_110,f,0.9251876,100,7861,0.87,3966,473,117,102,4,...,1,0,0,0,0,3,niche3,C,4,B1
c_2_100_117,f,0.9999998,100,9632,0.95,1565,494,110,116,22,...,0,0,0,0,0,7,niche5,C,4,B1
c_2_100_12,f,0.9999126,100,15877,0.53,3267,55,206,110,0,...,0,0,0,0,0,11,niche5,C,4,B1
c_2_100_126,f,0.9999994,100,9473,0.34,4222,578,65,193,585,...,2,0,0,0,0,2,niche3,C,4,B1
